In [1]:
# SPMF - try to get most occurring sequences
from spmf import Spmf
import pandas as pd
from text_cleaner import *
from tqdm import tqdm
import itertools

archetype_list = ['artist',
                 'caregiver',
                 'everyman',
                 'explorer',
                 'guru',
                 'hero',
                 'innocent',
                 'jester',
                 'magician',
                 'rebel',
                 'ruler',
                 'seducer']

## Loading the dataset and cleaning the text

In [2]:
# Load the Twitter dataset
twitter_df = pd.read_csv('tweets_06_03_2021.csv', index_col=0)

# Print the head of the loaded dataset
twitter_df.head()

,_id,tweet_text,username,created_at,timestamp,archetype
0,5f9f1c36b38e10f823bf2cdc,"@AndruEdwards The hard work has paid off, this...",LEGO_Group,2020-11-01 19:32:05.000,NaN,artist
1,5f9f1c36b38e10f823bf2cdd,@soosupersam A great way to surprise your love...,LEGO_Group,2020-11-01 19:09:40.000,NaN,artist
2,5f9f1c36b38e10f823bf2cde,"You can now just bring the fun home, and reliv...",LEGO_Group,2020-11-01 14:00:36.000,NaN,artist
3,5f9f1c36b38e10f823bf2cdf,@at_knb Happy birthday to the master builder! ...,LEGO_Group,2020-10-31 17:16:57.000,NaN,artist
4,5f9f1c36b38e10f823bf2ce0,@dizunatsu 😀😀,LEGO_Group,2020-10-31 15:18:50.000,NaN,artist


In [3]:
# Clean-up the texts
twitter_df["cleaned_text"] = twitter_df["tweet_text"].apply(lambda x: clean_up_text(x))

# Tokenize the cleaned texts
twitter_df["cleaned_text"] = twitter_df["cleaned_text"].apply(lambda x: nltk.word_tokenize(x))

# Remove the stopwords
twitter_df["cleaned_text"] = twitter_df["cleaned_text"].apply(lambda x: remove_stopwords(x))

# Drop the rows with empty 'cleaned_text' field
twitter_df = twitter_df.drop(twitter_df[twitter_df['cleaned_text'].map(len) < 1].index)

# Print the new head of the dataset
twitter_df.head()

,_id,tweet_text,username,created_at,timestamp,archetype,cleaned_text
0,5f9f1c36b38e10f823bf2cdc,"@AndruEdwards The hard work has paid off, this...",LEGO_Group,2020-11-01 19:32:05.000,NaN,artist,"[hard, work, paid, awesome]"
1,5f9f1c36b38e10f823bf2cdd,@soosupersam A great way to surprise your love...,LEGO_Group,2020-11-01 19:09:40.000,NaN,artist,"[great, way, surprise, loved, one]"
2,5f9f1c36b38e10f823bf2cde,"You can now just bring the fun home, and reliv...",LEGO_Group,2020-11-01 14:00:36.000,NaN,artist,"[bring, fun, home, relive, favorite, childhood..."
3,5f9f1c36b38e10f823bf2cdf,@at_knb Happy birthday to the master builder! ...,LEGO_Group,2020-10-31 17:16:57.000,NaN,artist,"[happy, birthday, master, builder, hope, magic..."
6,5f9f1c36b38e10f823bf2ce2,@Ranchie This is the way! 😀,LEGO_Group,2020-10-31 15:16:26.000,NaN,artist,[way]


## Testing SPMF on 'artist' archetype

In [4]:
# Example 1 - try to get the most occurring words in the 'artist' archetype subset
# Extract all the tweets for the 'artist' archetype
artist_df = twitter_df.cleaned_text[twitter_df["archetype"] == "artist"]

# Reset the index of the subset
artist_df = artist_df.reset_index(drop=True)

# Print the head of the subset
artist_df.head()

0                          [hard, work, paid, awesome]
1                   [great, way, surprise, loved, one]
2    [bring, fun, home, relive, favorite, childhood...
3    [happy, birthday, master, builder, hope, magic...
4                                                [way]
Name: cleaned_text, dtype: object

In [5]:
# Create a list of sentences
artist_list = [" ".join(row) for row in artist_df.tolist()]

In [8]:
# SPMF - get the most frequent sequences
spmf = Spmf("PrefixSpan", input_direct=artist_list,
            output_filename="output.txt", arguments=[0.0003, 3], input_type="text")
spmf.run()
print(spmf.to_pandas_dataframe(pickle=True))
spmf.to_csv("output.csv")

>/mnt/HDD_Linux/Praca_magisterska/jupyter_notebooks/data_mining/spmf.jar
Converting TEXT to SPMF format.
Conversion completed.
=============  PREFIXSPAN 0.99-2016 - STATISTICS =============
 Total time ~ 524 ms
 Frequent sequences count : 219786
 Max memory (mb) : 139.26904296875
 minsup = 4 sequences.
 Pattern count : 219786

Post-processing to show result in terms of string values.
Post-processing completed.

                              pattern  sup
0                              [hard]  137
1                      [hard, placed]    5
2                        [hard, work]   18
3                 [hard, work, every]    4
4              [hard, work, everyone]    4
...                               ...  ...
219781              [ds, later, year]    4
219782                     [ds, year]    4
219783                  [swatchxmoma]    6
219784  [swatchxmoma, swatchlovesart]    6
219785                           [bp]    4

[219786 rows x 2 columns]


# Calculating most frequent sequences for all of the archetypes

In [9]:
full_df = pd.DataFrame(columns=['chunk'] + archetype_list)

# Iterate over archetypes
for archetype in tqdm(archetype_list):
    # Extract all the tweets for the 'artist' archetype
    tmp_df = twitter_df.cleaned_text[twitter_df["archetype"] == archetype]

    # Reset the index of the subset
    tmp_df = tmp_df.reset_index(drop=True)
    
    # Calculate number of words, number of two-word and three-word combinations
    unique_words = set()
    unique_two_words = set()
    unique_three_words = set()
    
    for row in tmp_df.tolist():
        unique_words.update(row)
        unique_two_words.update(itertools.permutations(row, 2))
        unique_three_words.update(itertools.permutations(row, 3))
    
    cnt_single_word = len(unique_words)
    cnt_two_words = len(unique_two_words)
    cnt_three_words = len(unique_three_words)
    
    # Create a list of sentences
    tmp_list = [" ".join(row) for row in tmp_df.tolist()]
    
    # SPMF - get the most frequent sequences
    spmf = Spmf("PrefixSpan", input_direct=tmp_list,
                output_filename=f"sequence_files_four/output_{archetype}.txt", arguments=[0.0003, 3], input_type="text")
    spmf.run()
    
    spmf = spmf.to_pandas_dataframe(pickle=True)
    
    # Get the TF
    spmf.sup[spmf.pattern.map(len) == 1] = spmf.sup[spmf.pattern.map(len) == 1].apply(lambda x: float(x / cnt_single_word))
    spmf.sup[spmf.pattern.map(len) == 2] = spmf.sup[spmf.pattern.map(len) == 2].apply(lambda x: float(x / cnt_two_words)) 
    spmf.sup[spmf.pattern.map(len) == 3] = spmf.sup[spmf.pattern.map(len) == 3].apply(lambda x: float(x / cnt_three_words)) 
    
    print(spmf)
    spmf.to_csv(f"sequence_files/output_{archetype}.csv")

  0%|          | 0/12 [00:00<?, ?it/s]

>/mnt/HDD_Linux/Praca_magisterska/jupyter_notebooks/data_mining/spmf.jar
Converting TEXT to SPMF format.
Conversion completed.
=============  PREFIXSPAN 0.99-2016 - STATISTICS =============
 Total time ~ 461 ms
 Frequent sequences count : 219786
 Max memory (mb) : 183.41357421875
 minsup = 4 sequences.
 Pattern count : 219786

Post-processing to show result in terms of string values.
Post-processing completed.



<ipython-input-9-336d2b54d506>:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spmf.sup[spmf.pattern.map(len) == 1] = spmf.sup[spmf.pattern.map(len) == 1].apply(lambda x: float(x / cnt_single_word))


                              pattern           sup
0                              [hard]  1.221796e-02
1                      [hard, placed]  6.083746e-06
2                        [hard, work]  2.190149e-05
3                 [hard, work, every]  2.133147e-07
4              [hard, work, everyone]  2.133147e-07
...                               ...           ...
219781              [ds, later, year]  2.133147e-07
219782                     [ds, year]  4.866997e-06
219783                  [swatchxmoma]  5.350932e-04
219784  [swatchxmoma, swatchlovesart]  7.300496e-06
219785                           [bp]  3.567288e-04

[219786 rows x 2 columns]


  8%|▊         | 1/12 [00:12<02:20, 12.76s/it]

...

 92%|█████████▏| 11/12 [02:55<00:15, 15.76s/it]

>/mnt/HDD_Linux/Praca_magisterska/jupyter_notebooks/data_mining/spmf.jar
Converting TEXT to SPMF format.
Conversion completed.
=============  PREFIXSPAN 0.99-2016 - STATISTICS =============
 Total time ~ 384 ms
 Frequent sequences count : 540601
 Max memory (mb) : 114.53091430664062
 minsup = 2 sequences.
 Pattern count : 540601

Post-processing to show result in terms of string values.
Post-processing completed.



<ipython-input-9-336d2b54d506>:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spmf.sup[spmf.pattern.map(len) == 1] = spmf.sup[spmf.pattern.map(len) == 1].apply(lambda x: float(x / cnt_single_word))


                                pattern           sup
0                             [holiday]  1.682073e-02
1                    [holiday, holiday]  2.112038e-05
2             [holiday, holiday, boxes]  2.199042e-07
3               [holiday, holiday, box]  2.199042e-07
4            [holiday, holiday, spirit]  2.199042e-07
...                                 ...           ...
540596         [galgadot, tiffanyandco]  5.280096e-06
540597                [tiffanybluebook]  3.030762e-04
540598  [tiffanybluebook, tiffanyandco]  5.280096e-06
540599                   [goldenglobes]  6.061524e-04
540600     [goldenglobes, tiffanyandco]  7.920144e-06

[540601 rows x 2 columns]


100%|██████████| 12/12 [03:14<00:00, 16.17s/it]


In [11]:
# Merge all files into a single dataframe
total_df = pd.DataFrame(columns=["pattern"] + archetype_list)
for archetype in tqdm(archetype_list):    
    # Read file and merge it with the other ones
    total_df = pd.concat([total_df, pd.read_csv(f"sequence_files/output_{archetype}.csv", sep=",", index_col=0).rename(columns={"sup": archetype})], ignore_index=True)

100%|██████████| 12/12 [00:14<00:00,  1.17s/it]


In [12]:
# Print dataframe
total_df

,pattern,artist,caregiver,everyman,explorer,guru,hero,innocent,jester,magician,rebel,ruler,seducer
0,['hard'],1.221796e-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"['hard', 'placed']",6.083746e-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"['hard', 'work']",2.190149e-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"['hard', 'work', 'every']",2.133147e-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"['hard', 'work', 'everyone']",2.133147e-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4150610,"['galgadot', 'tiffanyandco']",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.2801e-06
4150611,['tiffanybluebook'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000303076
4150612,"['tiffanybluebook', 'tiffanyandco']",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.2801e-06
4150613,['goldenglobes'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000606152


In [13]:
aggregate_func = {
    "pattern": "first",
    "artist": "sum",
    "caregiver": "sum",
    "everyman": "sum",
    "explorer": "sum",
    "guru": "sum",
    "hero": "sum",
    "innocent": "sum",
    "jester": "sum",
    "magician": "sum",
    "rebel": "sum",
    "ruler": "sum",
    "seducer": "sum"
}
total_df = total_df.groupby("pattern").aggregate(aggregate_func)
total_df = total_df.reset_index(drop=True)

KeyboardInterrupt: 

In [ ]:
# Print the full DataFrame
total_df

In [ ]:
# Save the current DataFrame
total_df.to_csv("sequence_files_four/phrase_frequency_no_df.csv")

In [ ]:
# Calculate document frequency for every archetype
import math
import ast
import dask.dataframe as dd
from dask.diagnostics import ProgressBar

def get_doc_freq(phrase, sup, dataset):
    phrase_cnt = 0
    for line in dataset:
        if tuple(phrase) in line:
            phrase_cnt += 1
    return sup * math.log(len(dataset) / (phrase_cnt + 1))
    

for archetype in archetype_list:
    print(f"Archetype {archetype}:")
    tmp_df = pd.read_csv(f"sequence_files/output_{archetype}.csv").set_index("Unnamed: 0")
    tmp_df["pattern"] = tmp_df["pattern"].apply(lambda x: ast.literal_eval(x))
    
    tmp_df = dd.from_pandas(tmp_df, 10000)
    
    twitter_subset = twitter_df.cleaned_text[twitter_df["archetype"] == archetype]
    
    # Select non-zero elements and calculate DF for every element
    with ProgressBar():
        tmp_df["sup"] = tmp_df.apply(lambda row: get_doc_freq(row["pattern"], row["sup"], twitter_subset), axis=1, meta=(float)).compute()

    tmp_df.to_csv(f"sequence_files_four/output_{archetype}_df.csv")

Archetype artist:
[########################################] | 100% Completed | 15min 32.3s
Archetype caregiver:
[###########                             ] | 28% Completed | 11min 44.1s

In [ ]:
# Save the DataFrame with document frequency calculated
total_df.to_csv("sequence_files/phrase_frequency_with_df.csv")

In [ ]:
# Print the new DataFrame
total_df